In [57]:
import pandas as pd
import numpy as np
import json
import os
from collections import deque
from tqdm import tqdm
pd.set_option('display.max_rows', 1000+1)
pd.set_option('display.max_columns', 1000+1)


##### 1, Replace or remove outlier

In [2]:
# replace the value of the shift

data_dir = 'dataset/Godess_final_data/Godess_data/'
out_data_dir = 'dataset/Godess_final_data/Godess_data_rev/'
label_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_carbon_csv/'
connection_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_connection_dict/'
pdb_files = os.listdir(data_dir)

In [3]:
# from ananymous's comment
# labeled_all_1591.pdb.csv, row 6 in test-carbon, the C2 seems very wrong in this glycan for that major outlier
# 97.2 -> 79.4

# Use _hyb for 888 instead of _stat for train-carbon. Another anomaly that shows up just in the stat file.

# Etc......


def replace_values(df_pdb, shift_idx, atom_name, old_shift, new_shift):
    
    df_pdb_out = df_pdb.copy()
    
    df_pdb_out.loc[(df_pdb_out['Atom_name'] == atom_name) & 
                   (df_pdb_out['shift'] == old_shift) & 
                   (df_pdb_out.index == shift_idx), ['shift']] = new_shift
    
    return df_pdb_out

In [4]:
# replace values by whole monosacchaide
def replace_values_monosaccharide(df_pdb, old_shift_list, new_shift_list, Residual_accurate_name, trust_val, 
                                 carbon_list = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']):
    df_pdb_out = df_pdb.copy()
    assert(len(new_shift_list) == len(carbon_list))
    for i in range(len(old_shift_list)):
        
        idx0 = df_pdb_out.loc[(df_pdb_out['Residual_accurate_name'] == Residual_accurate_name) & 
                           (df_pdb_out['trust'] == trust_val) & 
                           (df_pdb_out['Atom_name'] == carbon_list[i]) & 
                           (df_pdb_out['shift'] == old_shift_list[i]), ['shift']].empty
        
        if idx0:
            print(old_shift_list)
            print(i, old_shift_list[i], Residual_accurate_name, trust_val, 'empty monosaccharide')
        
        
        df_pdb_out.loc[(df_pdb_out['Residual_accurate_name'] == Residual_accurate_name) & 
                       (df_pdb_out['trust'] == trust_val) & 
                       (df_pdb_out['Atom_name'] == carbon_list[i]) & 
                       (df_pdb_out['shift'] == old_shift_list[i]), ['shift']] = new_shift_list[i]
        
        idx1 = df_pdb_out.loc[(df_pdb_out['Residual_accurate_name'] == Residual_accurate_name) & 
                           (df_pdb_out['trust'] == trust_val) & 
                           (df_pdb_out['Atom_name'] == carbon_list[i]) & 
                           (df_pdb_out['shift'] == new_shift_list[i]), ['shift']].empty
        
        if idx1:
            print(old_shift_list)
            print(i, old_shift_list[i], Residual_accurate_name, trust_val, 'empty monosaccharide')
        
    return df_pdb_out
    
    

##### 2, Check abnormal shifts

In [5]:
pdb_name_list = []
atom_name_list = []
mono_name_list = []
shift_list = []
idx_list = []

##### 2.1 Check lineage

In [6]:
def check_and_reformulate_lineage(df_pdb):
    new_lineage_list = []
    for lineage in df_pdb['Lineage'].values:
        if (isinstance(lineage, float)) and ('.0' in str(lineage)):
            new_lineage = str(lineage).replace('.0', '')
            new_lineage_list.append(new_lineage)
        else:
            new_lineage_list.append(lineage)
    
    df_pdb_copy = df_pdb.copy()
    
    df_pdb_copy['Lineage'] = new_lineage_list
    
    return df_pdb_copy

##### 3, Add new features for root Me, Ser

In [7]:
initial_list = []
root_list = []
me_lineage_list = []
asn_lineage_list = []
ser_lineage_list = []

In [8]:
# old function, create interaction features by missing monosaccharide.
def create_root_effect_feature(df_pdb, root_residue = 'Me', num_interaction = 2):
    
    df_lineage_list = df_pdb['Lineage'].values
    df_residue_list = df_pdb['Residual_accurate_name'].values

    new_interaction_list = [root_residue + '_' + str(i) for i in range(1, 2 + 1)]
    new_interaction_list.insert(0, root_residue)
    
    df_interaction_list = np.repeat('no_interaction', len(df_lineage_list))

    idx = 0

    prev_root = 'Missing root'
    
    root_indicator = 0

    for i in range(len(df_lineage_list)):

        current_lineage = df_lineage_list[i]
        
        if isinstance(current_lineage, float):
            current_lineage = int(current_lineage)
            current_lineage = str(current_lineage)
            
        current_residue = df_residue_list[i]
        
        if current_residue == root_residue:

            df_interaction_list[i] = new_interaction_list[0]
            
            prev_root = current_lineage
            
            assert(prev_root == '')
            
            root_indicator = 1
            
        elif (len(current_lineage) == 1) and (root_indicator):
            
            prev_root = current_lineage
            
            df_interaction_list[i] = new_interaction_list[1]

        elif (len(current_lineage) == 3) and (current_lineage.split(',')[0] == prev_root) and (root_indicator):

            df_interaction_list[i] = new_interaction_list[2]
#             df_interaction_list[i] = 'no_interaction'
    
    df_pdb_copy = df_pdb.copy()
    
    column_name = root_residue + "_interaction"
    
    df_pdb_copy[column_name] = df_interaction_list
    
    return df_pdb_copy

##### 3.1, Revised Add new features for root Me, Ser, in distance, based on the Me interaction. 

In [9]:
# The connection information from the pdb file is cannot be fully trusted, some file have the patterns that A -> B, but B is not connected to A
def check_and_modify_graph(graph):
    """
    Checks if a graph satisfies the property that if node A is connected to node B, then node B is connected to node A.
    If the property is not satisfied, modifies the graph by adding the missing connections.
    
    Arguments:
    graph -- a dictionary representing a graph where the keys are the nodes and the values are lists of connected nodes
    
    Returns:
    The modified graph.
    """
    for node in graph:
        for neighbor in graph[node]:
            if node not in graph[neighbor]:
                graph[neighbor].append(node)
    return graph

In [10]:
# read in connection from step 2 (create linkage for pdb file)
def read_in_connection(connection_dir, pdb_name):
    
    name = pdb_name.split('labeled_all_')[1].split('.pdb')[0]
    
    connect_name = str(name) + '_connection.json'
    
    dict_f_path = os.path.join(connection_dir, connect_name)
    
    with open(dict_f_path) as dict_f_json:
        dict_f = json.load(dict_f_json)
    
    dict_f = {int(k):v for k,v in dict_f.items()}
    
    dict_f = check_and_modify_graph(dict_f)
    
    return dict_f

In [11]:
# given target root, for each atom, find the shortest distance to the root atom (Carbon) using BFS
def bfs_shortest_path(graph, start_node, end_node):
    """
    Compute the shortest path from start_node to end_node in graph using Breadth-First Search.
    
    Arguments:
    graph -- a dictionary representing the graph, where keys are nodes and values are lists of neighbors
    start_node -- the node to start the search from
    end_node -- the node to search for
    
    Returns:
    A list containing the shortest path as a list of nodes.
    """
    # Initialize the visited set, the queue, and the parent dictionary with start_node as the only node visited.
    visited = set([start_node])
    queue = deque([(start_node, [])])
    parent = {start_node: None}
    
    while queue:
        # Dequeue the next node and its path.
        current_node, path = queue.popleft()
        
        # If we've found the end_node, return its path.
        if current_node == end_node:
            return path + [end_node]
        
        # Loop through the neighbors of the current_node and enqueue them if they haven't been visited.
        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [current_node]))
                parent[neighbor] = current_node
    
    # If we've gone through all the nodes and haven't found the end_node, return an empty path.
    return []

In [12]:
# create atom distance feature for root components, 
#1, distance, 2, categorical value with number of bounds as threshold, # exact shortest atom path
def create_atom_distance_feature_root(connection_dir, pdb_name, df_pdb, root, threshold):
    
    if root in df_pdb['Residual_accurate_name'].values:
    
        dict_f = read_in_connection(connection_dir, pdb_name)

        atom_num_root = df_pdb.loc[df_pdb['Residual_accurate_name'] == root, :]['Atom_num'].values

        all_atom_num_list = df_pdb['Atom_num'].values

        all_atom_name_list = df_pdb['Atom_name'].values

        residue_interaction_list = np.repeat(999, len(df_pdb))

        max_atom = np.max(atom_num_root)
#         print(atom_num_root)
        connect_atom_outside_list = []


        for i in atom_num_root:
            current_connect_atom = dict_f[i]
            for j in current_connect_atom:
                
                # current atom not belongs to the root group, then it should connect to the other group
                
                if j not in atom_num_root:
                    connect_atom_outside_list.append(i)
#         print(connect_atom_outside_list)
        print(connect_atom_outside_list)
        assert(len(connect_atom_outside_list) == 1)
        
        target_distance_list = []

        target_distance_atom_list = []


        for k in connect_atom_outside_list:

            for l in all_atom_num_list:

                current_shortest_path = bfs_shortest_path(dict_f, k, l)
                
                # set distance to 0 for atoms within the same root group
                if k == l:
                    target_distance_list.append(0)

                elif l in atom_num_root:

                    target_distance_list.append(0)
                # set the distance to other atoms 
                else:
                    target_distance_list.append(len(current_shortest_path) - 1)


                current_shortest_path_atom_type = [all_atom_name_list[m - 1] for m in current_shortest_path]

                target_distance_atom_list.append(current_shortest_path_atom_type)

        target_distance_list = np.array(target_distance_list)

        target_distance_list_threshold = target_distance_list.copy()

        target_distance_list_threshold[(target_distance_list_threshold <= threshold) & (target_distance_list_threshold > 0)] = 1

        target_distance_list_threshold[target_distance_list_threshold > threshold] = 2

        df_pdb_copy = df_pdb.copy()

        df_pdb_copy[root + '_atom_distance'] = target_distance_list

        df_pdb_copy[root + '_atom_distance_threshold'] = target_distance_list_threshold

        df_pdb_copy[root + '_atom_path'] = target_distance_atom_list
    
    else:
        df_pdb_copy = df_pdb.copy()
        
        df_pdb_copy[root + '_atom_distance'] = 999

        df_pdb_copy[root + '_atom_distance_threshold'] = 2

        df_pdb_copy[root + '_atom_path'] = [['No ' + root + ' included'] for _ in range(len(df_pdb))]
        
        
    
    return df_pdb_copy


##### 4, Add new features for root Ac, Gc, S. residue level.

In [13]:
ac_list = []
gc_list = []
s_list = []
all_ac_list = []

In [14]:
# old function, create Ac/S/Gc feature by monosaccharide names and distance
def create_residue_effect_feature(df_pdb, df_label, target_residue = 'S'):

    df_pdb_copy = df_pdb.copy()

    df_effect_list = np.repeat('no_effect', len(df_pdb_copy))

    for i in range(len(df_label)):
        current_linkage = df_label.loc[i, :]['Linkage']
        
        if isinstance(current_linkage, float):
            current_linkage = int(current_linkage)
            current_linkage = str(current_linkage)
        
        current_residue = df_label.loc[i, :]['Residue']
        if (current_residue == target_residue):
            
            current_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == current_linkage].index
            
            if len(df_label.loc[df_label['Linkage'] == current_linkage]) == 1:
            
                df_effect_list[current_index] = target_residue

            attached_linkage = current_linkage[0:-2]

            attached_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage].index

            df_effect_list[attached_index] = target_residue + '_1'

    column_name = target_residue + '_effect'

    df_pdb_copy[column_name] = df_effect_list
    
    return df_pdb_copy

##### 4.1 Identify Ac components in atom level, 

Most Ac components do not have a specific residue label in the pdb file, therefore, we can only identify them by atom names. Also, the Ac compoenets in the labeled csv file contain the branch information, this can help for the atom level idenfication. 

##### The connection file failed in some files like 251, atom A to B but no B to A

In [15]:
# identify Ac components with the help of connection information, branched info from labedled csv and pdb residue names
def identify_AC_components(df_pdb, df_pdb_name, df_label, connection_dir, j):

    dict_f = read_in_connection(connection_dir, df_pdb_name)

    df_pdb_copy = df_pdb.copy()

    df_label = df_label

    Ac_components_list = np.repeat(0, len(df_pdb_copy))

    for i in range(len(df_label)):
        current_linkage = df_label.loc[i, :]['Linkage']

        if isinstance(current_linkage, float):
            current_linkage = int(current_linkage)
            current_linkage = str(current_linkage)

        current_residue = df_label.loc[i, :]['Residue']

        if current_residue == 'Ac':

            attached_linkage = current_linkage[0:-2]

            attached_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage].index

            attached_atom_name = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage, :]['Atom_name'].values

            if 'C11' in attached_atom_name:
                c11_index_list = np.where(attached_atom_name == 'C11')[0]

                # Attached only one monosaccharide
                if len(c11_index_list) == 1:

                    # c11 index in csv
                    c11_index = attached_index[c11_index_list[0]]

                    # c11 index in atom names
                    c11_atom_num = df_pdb_copy.loc[c11_index, :]['Atom_num']

                    c11_connection_list = dict_f[c11_atom_num]

                    Ac_residue_index_list = [c11_atom_num]

                    for atom_num in c11_connection_list:

                        current_atom_connect = dict_f[atom_num]
                        
                        # in case of the connection file failed
                        if c11_atom_num in current_atom_connect:
                            current_atom_connect.remove(c11_atom_num)

                        # identify the Oxx
                        if len(current_atom_connect) == 0:

                            Ac_residue_index_list.append(atom_num)

                        else:
                            len_atom_connect = 0
                            for atom_num_connect in current_atom_connect:


                                atom_num_connect_atom_list = dict_f[atom_num_connect]
                                
                                if atom_num in atom_num_connect_atom_list:

                                    atom_num_connect_atom_list.remove(atom_num)
    #                             print(atom_num_connect, atom_num_connect_atom_list, atom_num)

                                len_atom_connect += len(atom_num_connect_atom_list)

                            # identify the C21, and three HXXs 
                            if len_atom_connect == 0:

                                Ac_residue_index_list.append(atom_num)
                                Ac_residue_index_list.extend(dict_f[atom_num])

                    Ac_residue_index_list = df_pdb_copy.loc[df_pdb_copy['Atom_num'].isin(Ac_residue_index_list)].index
                    Ac_components_list[Ac_residue_index_list] = 1
    #                     print(atom_num, current_atom_connect)
                else:
                    print(df_pdb_name, j, 'contain more than one C11')


            else:

                print(df_pdb_name, j, current_residue, 'the attached monosaccharides does not contain a C11')

                current_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == current_linkage].index

                current_three_letter_residual = df_pdb_copy.loc[current_index, :]['Residual_name'].values

                # the attached monosaccharide 
                assert(np.all(current_three_letter_residual == 'ACY'))

    # some ACs are behave like are separate components
    if not df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].empty:
        ACY_index = df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].index
        Ac_components_list[ACY_index] = 1

    df_pdb_copy['Ac_component'] = Ac_components_list
    
    return df_pdb_copy

##### 4.2 Based on the identified Acs, for each Ac, find the shortest distance between the current Ac 

In [16]:
def create_atom_distance_feature_Ac(df_pdb, pdb_name, threshold = 5):

    df_pdb_copy = df_pdb.copy()

    dict_f = read_in_connection(connection_dir, pdb_name)

    df_c11 = df_pdb_copy.loc[(df_pdb_copy['Ac_component'] == 1) & 
                             ((df_pdb_copy['Atom_name'] == 'C11') | 
                              (df_pdb_copy['Atom_name'] == 'C1'))]
    if not df_c11.empty:
        df_c11.index = range(len(df_c11))

        # since there could be multiple Acs in a glycan, each glycan corresponds to a glycan list
        # therefore, need to first retrieve list of glycan distance list

        all_Ac_distance_list_list = []

        all_Ac_distance_threshold_list_list = []

        all_Ac_atom_name_list_list_list = []

        all_atom_num_list = df_pdb_copy['Atom_num'].values

        all_atom_name_list = df_pdb_copy['Atom_name'].values

        for i in range(len(df_c11)):

            current_target_distance_list = []

            current_target_atom_name_list_list = []

            current_atom_num = df_c11.loc[i, :]['Atom_num']

            for j in all_atom_num_list:

                current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)

                current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]

                # 3, list of list of atom paths, index + 1 = atom num
                current_target_atom_name_list_list.append(current_atom_name_path)

                # 2, Assign distance 
                if current_atom_num == j:

                    current_target_distance_list.append(0)

                # atom with in the same Ac component labeled as 0
                elif ((len(current_atom_num_path) - 1) <= 2) and (df_pdb_copy.loc[j-1, :]['Ac_component'] == 1):

                    current_target_distance_list.append(0)

                else:
                    current_target_distance_list.append(len(current_atom_num_path) - 1)

            # 1, Assign categorical value by threshold as distance bound
            current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

            current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

            current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2

            current_target_distance_list_threshold = list(current_target_distance_list_threshold)

            # list of list stores distance to each Acs in a glycan
        #     print(current_target_distance_list)
            all_Ac_distance_list_list.append(current_target_distance_list)

            # this is not used in this function, but may be used in the future if other method is developed
            all_Ac_distance_threshold_list_list.append(current_target_distance_list_threshold)

            all_Ac_atom_name_list_list_list.append(current_target_atom_name_list_list)

        all_Ac_distance_list_list = np.array(all_Ac_distance_list_list)

        # find the minimum distance
        new_all_index = all_Ac_distance_list_list.argmin(axis=0)

        minimum_all_Ac_distance_list_list = [all_Ac_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

        # find the corresponding minimum threshold
        minimum_all_Ac_distance_list_list_threshold = np.array(minimum_all_Ac_distance_list_list.copy())

        minimum_all_Ac_distance_list_list_threshold[(minimum_all_Ac_distance_list_list_threshold <= threshold) & (minimum_all_Ac_distance_list_list_threshold > 0)] = 1

        minimum_all_Ac_distance_list_list_threshold[minimum_all_Ac_distance_list_list_threshold > threshold] = 2

        minimum_all_Ac_distance_list_list_threshold = list(minimum_all_Ac_distance_list_list_threshold)

        # find the corresponding minimum atom names

        minimum_all_Ac_atom_name_list_list_list = [all_Ac_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]


        df_pdb_copy['Ac' + '_min_atom_distance'] = minimum_all_Ac_distance_list_list

        df_pdb_copy['Ac' + '_min_atom_distance_threshold'] = minimum_all_Ac_distance_list_list_threshold

        df_pdb_copy['Ac' + '_min_atom_path'] = minimum_all_Ac_atom_name_list_list_list
    
    else:
        # not containing any Ac components
        df_pdb_copy['Ac' + '_min_atom_distance'] = 999

        df_pdb_copy['Ac' + '_min_atom_distance_threshold'] = 2

        df_pdb_copy['Ac' + '_min_atom_path'] = [['No ' + '_Ac_' + ' included'] for _ in range(len(df_pdb))]
    
    return df_pdb_copy

##### 4.3 Indentiy the Sulfur components from the pdb file, then create distance by atom path distances, similar process as Ac

In [17]:
def create_atom_distance_feature_S(df_pdb, df_pdb_name, df_label, threshold = 5):

    df_pdb_copy = df_pdb.copy()

    dict_f = read_in_connection(connection_dir, df_pdb_name)

    df_s = df_pdb_copy.loc[(df_pdb_copy['Residual_name'] == 'SO4') & 
                             (df_pdb_copy['Atom_type'] == 'S')]
    df_s.index = range(len(df_s))

    if (not df_s.empty) and ('S' in df_label['Residue'].values):

        all_S_distance_list_list = []

        all_S_distance_threshold_list_list = []

        all_S_atom_name_list_list_list = []

        all_atom_num_list = df_pdb_copy['Atom_num'].values

        all_atom_name_list = df_pdb_copy['Atom_name'].values

        for i in range(len(df_s)):

            current_target_distance_list = []

            current_target_atom_name_list_list = []

            current_atom_num = df_s.loc[i, :]['Atom_num']

            for j in all_atom_num_list:

                current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)

                current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]

                # 3, list of list of atom paths, index + 1 = atom num
                current_target_atom_name_list_list.append(current_atom_name_path)

                # 2, Assign distance 
                if current_atom_num == j:

                    current_target_distance_list.append(0)

                # atom with in the same S component labeled as 0
                elif ((len(current_atom_num_path) - 1) <= 2) and (df_pdb_copy.loc[j-1, :]['Residual_accurate_name'] == 'S'):

                    current_target_distance_list.append(0)

                else:
                    current_target_distance_list.append(len(current_atom_num_path) - 1)


             # 1, Assign categorical value by threshold as distance bound
            current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

            current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

            current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2

            current_target_distance_list_threshold = list(current_target_distance_list_threshold)

            # list of list stores distance to each Ss in a glycan
    #         print(current_target_distance_list)

            all_S_distance_list_list.append(current_target_distance_list)

            # this is not used in this function, but may be used in the future if other method is developed
            all_S_distance_threshold_list_list.append(current_target_distance_list_threshold)

            all_S_atom_name_list_list_list.append(current_target_atom_name_list_list)


        all_S_distance_list_list = np.array(all_S_distance_list_list)

        # find the minimum distance
        new_all_index = all_S_distance_list_list.argmin(axis=0)

        minimum_all_S_distance_list_list = [all_S_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

        # find the corresponding minimum threshold
        minimum_all_S_distance_list_list_threshold = np.array(minimum_all_S_distance_list_list.copy())

        minimum_all_S_distance_list_list_threshold[(minimum_all_S_distance_list_list_threshold <= threshold) & (minimum_all_S_distance_list_list_threshold > 0)] = 1

        minimum_all_S_distance_list_list_threshold[minimum_all_S_distance_list_list_threshold > threshold] = 2

        minimum_all_S_distance_list_list_threshold = list(minimum_all_S_distance_list_list_threshold)

        # find the corresponding minimum atom names

        minimum_all_S_atom_name_list_list_list = [all_S_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]


        df_pdb_copy['S' + '_min_atom_distance'] = minimum_all_S_distance_list_list

        df_pdb_copy['S' + '_min_atom_distance_threshold'] = minimum_all_S_distance_list_list_threshold

        df_pdb_copy['S' + '_min_atom_path'] = minimum_all_S_atom_name_list_list_list

    else:
        # not containing any S components
        df_pdb_copy['S' + '_min_atom_distance'] = 999

        df_pdb_copy['S' + '_min_atom_distance_threshold'] = 2

        df_pdb_copy['S' + '_min_atom_path'] = [['No ' + '_S_' + ' included'] for _ in range(len(df_pdb))]
    
    return df_pdb_copy

##### 4.4, Indentiy the GC components from the pdb file, then create distance by atom path distances, similar process as Ac¶

In [18]:
def create_atom_distance_feature_Gc(df_pdb, pdb_name, threshold = 5):

    df_pdb_copy = df_pdb.copy()

    dict_f = read_in_connection(connection_dir, pdb_name)

    df_c11 = df_pdb_copy.loc[(df_pdb_copy['Residual_accurate_name'] == 'Gc') & 
                             ((df_pdb_copy['Atom_name'] == 'C11') | 
                              (df_pdb_copy['Atom_name'] == 'C1'))]
    if not df_c11.empty:
        df_c11.index = range(len(df_c11))

        # since there could be multiple Gcs in a glycan, eGch glycan corresponds to a glycan list
        # therefore, need to first retrieve list of glycan distance list

        all_Gc_distance_list_list = []

        all_Gc_distance_threshold_list_list = []

        all_Gc_atom_name_list_list_list = []

        all_atom_num_list = df_pdb_copy['Atom_num'].values

        all_atom_name_list = df_pdb_copy['Atom_name'].values

        for i in range(len(df_c11)):

            current_target_distance_list = []

            current_target_atom_name_list_list = []

            current_atom_num = df_c11.loc[i, :]['Atom_num']

            for j in all_atom_num_list:

                current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)

                current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]

                # 3, list of list of atom paths, index + 1 = atom num
                current_target_atom_name_list_list.append(current_atom_name_path)

                # 2, Assign distance 
                if current_atom_num == j:

                    current_target_distance_list.append(0)

                # atom with in the same Gc component labeled as 0
                elif ((len(current_atom_num_path) - 1) <= 3) and (df_pdb_copy.loc[j-1, :]['Residual_accurate_name'] == 'Gc'):

                    current_target_distance_list.append(0)

                else:
                    current_target_distance_list.append(len(current_atom_num_path) - 1)

            # 1, Assign categorical value by threshold as distance bound
            current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

            current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

            current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2

            current_target_distance_list_threshold = list(current_target_distance_list_threshold)

            # list of list stores distance to eGch Gcs in a glycan
        #     print(current_target_distance_list)
            all_Gc_distance_list_list.append(current_target_distance_list)

            # this is not used in this function, but may be used in the future if other method is developed
            all_Gc_distance_threshold_list_list.append(current_target_distance_list_threshold)

            all_Gc_atom_name_list_list_list.append(current_target_atom_name_list_list)

        all_Gc_distance_list_list = np.array(all_Gc_distance_list_list)

        # find the minimum distance
        new_all_index = all_Gc_distance_list_list.argmin(axis=0)

        minimum_all_Gc_distance_list_list = [all_Gc_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

        # find the corresponding minimum threshold
        minimum_all_Gc_distance_list_list_threshold = np.array(minimum_all_Gc_distance_list_list.copy())

        minimum_all_Gc_distance_list_list_threshold[(minimum_all_Gc_distance_list_list_threshold <= threshold) & (minimum_all_Gc_distance_list_list_threshold > 0)] = 1

        minimum_all_Gc_distance_list_list_threshold[minimum_all_Gc_distance_list_list_threshold > threshold] = 2

        minimum_all_Gc_distance_list_list_threshold = list(minimum_all_Gc_distance_list_list_threshold)

        # find the corresponding minimum atom names

        minimum_all_Gc_atom_name_list_list_list = [all_Gc_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]


        df_pdb_copy['Gc' + '_min_atom_distance'] = minimum_all_Gc_distance_list_list

        df_pdb_copy['Gc' + '_min_atom_distance_threshold'] = minimum_all_Gc_distance_list_list_threshold

        df_pdb_copy['Gc' + '_min_atom_path'] = minimum_all_Gc_atom_name_list_list_list
    
    else:
        # not containing any Gc components
        df_pdb_copy['Gc' + '_min_atom_distance'] = 999

        df_pdb_copy['Gc' + '_min_atom_distance_threshold'] = 2

        df_pdb_copy['Gc' + '_min_atom_path'] = [['No ' + '_Gc_' + ' included'] for _ in range(len(df_pdb))]
    
    return df_pdb_copy

##### 4.5, Indentiy the Me components from the pdb file, then create distance by atom path distances, similar process as Ac¶

In [19]:
def create_atom_distance_feature_Me(df_pdb, pdb_name, threshold = 4):

    df_pdb_copy = df_pdb.copy()

    dict_f = read_in_connection(connection_dir, pdb_name)
    
    # connection parts
    df_c11 = df_pdb_copy.loc[(df_pdb_copy['Residual_accurate_name'] == 'Me') & 
                             ((df_pdb_copy['Atom_name'] == 'C11') | 
                              (df_pdb_copy['Atom_name'] == 'C1'))]
    if not df_c11.empty:
        df_c11.index = range(len(df_c11))

        # since there could be multiple Gcs in a glycan, eGch glycan corresponds to a glycan list
        # therefore, need to first retrieve list of glycan distance list

        all_Gc_distance_list_list = []

        all_Gc_distance_threshold_list_list = []

        all_Gc_atom_name_list_list_list = []

        all_atom_num_list = df_pdb_copy['Atom_num'].values

        all_atom_name_list = df_pdb_copy['Atom_name'].values

        for i in range(len(df_c11)):

            current_target_distance_list = []

            current_target_atom_name_list_list = []

            current_atom_num = df_c11.loc[i, :]['Atom_num']

            for j in all_atom_num_list:

                current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)

                current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]

                # 3, list of list of atom paths, index + 1 = atom num
                current_target_atom_name_list_list.append(current_atom_name_path)

                # 2, Assign distance 
                if current_atom_num == j:

                    current_target_distance_list.append(0)

                # atom with in the same Gc component labeled as 0
                elif ((len(current_atom_num_path) - 1) <= 2) and (df_pdb_copy.loc[j-1, :]['Residual_accurate_name'] == 'Me'):

                    current_target_distance_list.append(0)

                else:
                    current_target_distance_list.append(len(current_atom_num_path) - 1)

            # 1, Assign categorical value by threshold as distance bound
            current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

            current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

            current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2

            current_target_distance_list_threshold = list(current_target_distance_list_threshold)

            # list of list stores distance to eGch Gcs in a glycan
        #     print(current_target_distance_list)
            all_Gc_distance_list_list.append(current_target_distance_list)

            # this is not used in this function, but may be used in the future if other method is developed
            all_Gc_distance_threshold_list_list.append(current_target_distance_list_threshold)

            all_Gc_atom_name_list_list_list.append(current_target_atom_name_list_list)

        all_Gc_distance_list_list = np.array(all_Gc_distance_list_list)

        # find the minimum distance
        new_all_index = all_Gc_distance_list_list.argmin(axis=0)

        minimum_all_Gc_distance_list_list = [all_Gc_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

        # find the corresponding minimum threshold
        minimum_all_Gc_distance_list_list_threshold = np.array(minimum_all_Gc_distance_list_list.copy())

        minimum_all_Gc_distance_list_list_threshold[(minimum_all_Gc_distance_list_list_threshold <= threshold) & (minimum_all_Gc_distance_list_list_threshold > 0)] = 1

        minimum_all_Gc_distance_list_list_threshold[minimum_all_Gc_distance_list_list_threshold > threshold] = 2

        minimum_all_Gc_distance_list_list_threshold = list(minimum_all_Gc_distance_list_list_threshold)

        # find the corresponding minimum atom names

        minimum_all_Gc_atom_name_list_list_list = [all_Gc_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]


        df_pdb_copy['Me' + '_min_atom_distance'] = minimum_all_Gc_distance_list_list

        df_pdb_copy['Me' + '_min_atom_distance_threshold'] = minimum_all_Gc_distance_list_list_threshold

        df_pdb_copy['Me' + '_min_atom_path'] = minimum_all_Gc_atom_name_list_list_list
    
    else:
        # not containing any Gc components
        df_pdb_copy['Me' + '_min_atom_distance'] = 999

        df_pdb_copy['Me' + '_min_atom_distance_threshold'] = 2

        df_pdb_copy['Me' + '_min_atom_path'] = [['No ' + '_Me_' + ' included'] for _ in range(len(df_pdb))]
    
    return df_pdb_copy

##### 5, Checkings, including save some test files, verify Gc components

##### 6, Create Neup atom names

In [20]:
def create_neup_feature(df_pdb):

    carbon_list = ['C1', 'C11', 'C2', 'C21', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

    hydrogen_list = ['H1', 'H11', 'H2', 'H3', 'H31', 'H4', 'H5', 'H51', 'H6', 'H61', 'H7', 'H8', 'H9', 'H91']

    df_pdb_copy = df_pdb.copy()
    
    new_atom_list = []
    for i in range(len(df_pdb)):
        current_atom_name = df_pdb_copy.loc[i, :]['Atom_name']
        current_residue_name = df_pdb_copy.loc[i, :]['Residual_accurate_name']

        if ('Neup' in current_residue_name) and ((current_atom_name in carbon_list) or \
                                                 (current_atom_name in hydrogen_list)):
            new_atom = 'Neup_' + current_atom_name
        else:
            new_atom = current_atom_name
        new_atom_list.append(new_atom)
    
    df_pdb_copy['New_Atom_name'] = new_atom_list
    
    return df_pdb_copy

##### 7, create Ac atom names based on step 6

In [21]:
def reformuate_atom_name_Ac(df_pdb):

    df_pdb_copy = df_pdb.copy()

    df_ac = df_pdb_copy.loc[df_pdb_copy['Ac_component'] == 1]

    for i in df_ac.index:

        df_pdb_copy.loc[i, ['New_Atom_name']] = df_pdb_copy.loc[i, ['New_Atom_name']] + '_Ac'
        
    return df_pdb_copy

In [22]:
def reformulate_atom_name_kdop(df_pdb, kdo_list = ['Kdo-ol', 'a-Kdop', 'b-Kdop']):
    idx_list = []
    for kdo in kdo_list:
        if kdo in df_pdb['Residual_accurate_name'].values:
            idx_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == kdo].index))
    
    df_pdb_copy = df_pdb.copy()
    df_pdb_copy.loc[idx_list, ['New_Atom_name']] = 'Kdo_' + df_pdb_copy.loc[idx_list, ['New_Atom_name']]
    
    return df_pdb_copy

In [23]:
def create_main_ring_shift(df_pdb, non_mono_components = ['17HOOle', 'Ac', 'Allyl', 'Asn', 'Bu', 'Bz', 'Caf', 'Cho', 
                                                            'Fer', 'Gallic', 'Gc', 'Mal', 'Me','Missing Monosaccharide', 
                                                             'P', 'Pr', 'S', 'Ser', 'myoIno', 'pCoum', 'myoIno', 'pCoum']):
    df_pdb_copy = df_pdb.copy()
    
    main_ring_shift = df_pdb_copy['shift'].values
    for i in range(len(df_pdb_copy)):
        current_mono = df_pdb.loc[i, :]['Residual_accurate_name']
        current_shift = df_pdb.loc[i, :]['shift']
        ac_indicator = df_pdb.loc[i, :]['Ac_component']
        if current_mono in non_mono_components:
            main_ring_shift[i] = -1.
            
        if ac_indicator == 1:
            main_ring_shift[i] = -1.
    
    df_pdb_copy['main_ring_shift'] = main_ring_shift
    
    return df_pdb_copy

### Implement all the above steps

In [24]:
for i in tqdm(range(len(pdb_files))):

# for i in [1]:
# for i in [730]:
# for i in [59]:

# for i in [7]:
# for i in [2]:
# for i in [1894]:

# for i in [517]:

# for i in [962]:

# for i in [179]:
# for i in [2398]:

    pdb_name = pdb_files[i]
    
    pdb_path = os.path.join(data_dir, pdb_name)
    
    out_pdb_path = os.path.join(out_data_dir, pdb_name)
    
    df_pdb = pd.read_csv(pdb_path)
    
    
    ##############################
    ### 1, replace, drop value ###
    ##############################
    
    
    # replace value of carbon outliers by hyb file
    
    if pdb_name == 'labeled_all_1591.pdb.csv':
        df_pdb = replace_values(df_pdb, 54, 'C2', 97.2, 79.4)
    
    if pdb_name == 'labeled_all_888.pdb.csv':
        df_pdb = replace_values_monosaccharide(df_pdb, [95.1, 70.6, 73.4, 79.5, 77.4, 175.9], 
                                              [95.1, 71.0, 73.4, 79.7, 77.2, 176.1], 'b-D-ManpA', 69)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [101.1, 75.8, 73.6, 69.5, 76.6, 176.5], 
                                              [101.2, 77.3, 74.3, 69.7, 76.9, 176.7], 'b-D-ManpA', 52)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [100.8, 77.8, 69.7, 70.5, 71.9, 62.0], 
                                              [100.7, 79.1, 69.4, 70.6, 72.2, 62.3], 'a-D-Galp', 48)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [105.2, 72.5, 73.8, 78.6, 74.0, 64.2], 
                                              [105.3, 72.2, 73.8, 78.5, 74.6, 64.5], 'b-D-Galp', 65)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [99.3, 77.4, 69.5, 66.4, 74.3, 170.0], 
                                              [100.4, 78.4, 70.7, 67.5, 74.3, 87.2], 'a-D-Manp', 79)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [103.3, 74.1, 75.1, 82.5, 76.6, 175.8], 
                                              [103.5, 74.1, 76.4, 80.3, 76.1, 174.5], 'b-D-GlcpA', 60)
        
        df_pdb = replace_values_monosaccharide(df_pdb, [101.9, 71.7, 73.7, 69.5, 73.2, 175.8], 
                                              [102.2, 71.4, 71.9, 70.3, 74.5, 177.6], 'a-D-ManpA', 59)
    
    if pdb_name == 'labeled_all_1417.pdb.csv':
        #1
        df_pdb = replace_values_monosaccharide(df_pdb, [95.9, 70.3, 77.6, 70.2, 36.6, 62.5], 
                                              [92.4, 73.1, 78.5, 70.3, 71.7, 62.4], 'a-D-Galp', 62)
        #2
        df_pdb = replace_values_monosaccharide(df_pdb, [96.6, 69.3, 70.4, 70.5, 72.1, 62.0], 
                                              [97.2, 69.5, 70.4, 70.6, 72.1, 62.3], 'a-D-Galp', 88)
        #3
        df_pdb = replace_values_monosaccharide(df_pdb, [103.6, 81.6, 76.9, 71.2, 76.9, 62.0], 
                                              [103.3, 81.3, 77.3, 70.9, 76.9, 61.8], 'b-D-Glcp', 74)
        #4
        df_pdb = replace_values_monosaccharide(df_pdb, [102.2, 57.0, 82.7, 70.2, 75.9, 66.0], 
                                              [102.2, 56.0, 82.6, 69.8, 76.3, 64.9], 'b-D-GlcpN', 54)
        #5
        df_pdb = replace_values_monosaccharide(df_pdb, [101.5, 56.7, 77.9, 71.1, 77.0, 62.3], 
                                              [101.5, 56.7, 81.1, 71.1, 77.0, 62.3], 'b-D-GlcpN', 49)
        #6
        df_pdb = replace_values_monosaccharide(df_pdb, [175.5, 96.9, 40.2, 67.4, 54.2, 70.7, 69.8, 73.1, 64.1], 
                                              [175.6, 100.8, 40.0, 73.1, 52.3, 70.8, 69.8, 73.1, 64.1], 
                                               'b-Neup', 31, 
                                               carbon_list=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])
        #7
        df_pdb = replace_values_monosaccharide(df_pdb, [174.5, 96.9, 37.1, 72.9, 50.8, 70.7, 69.8, 73.1, 64.1], 
                                              [175.7, 101.4, 39.0, 73.7, 52.2, 71.1, 69.6, 72.5, 64.3], 
                                               'b-Neup', 47, 
                                              carbon_list=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])
        #8
        df_pdb = replace_values_monosaccharide(df_pdb, [95.7, 68.7, 79.5, 69.9, 71.8, 61.8], 
                                              [99.8, 68.9, 79.5, 70.2, 72.1, 62.2], 'a-D-Galp', 57)
        #9
        df_pdb = replace_values_monosaccharide(df_pdb, [101.9, 71.1, 82.3, 73.4, 75.2, 61.1], 
                                              [102.5, 71.3, 81.3, 75.0, 75.9, 61.4], 'b-D-Manp', 53)
        #10
        df_pdb = replace_values_monosaccharide(df_pdb, [101.8, 69.8, 70.5, 70.7, 72.7, 62.2], 
                                              [101.8, 69.7, 70.4, 70.7, 72.4, 62.4], 'a-D-Galp', 74)
        #11
        df_pdb = replace_values_monosaccharide(df_pdb, [103.6, 79.0, 72.6, 71.4, 76.0, 65.3], 
                                              [103.1, 79.5, 73.5, 70.3, 75.5, 65.5], 'b-D-Galp', 57)
        #12
        df_pdb = replace_values_monosaccharide(df_pdb, [102.5, 82.8, 78.4, 70.1, 75.6, 175.0], 
                                              [102.6, 82.3, 76.7, 72.1, 75.9, 174.5], 'b-D-GlcpA', 64)
        #13
        df_pdb = replace_values_monosaccharide(df_pdb, [101.1, 75.1, 76.9, 73.4, 77.6, 176.3], 
                                      [103.6, 73.5, 76.5, 72.5, 75.9, 174.6], 'b-D-GlcpA', 57)
        
    
    # remove hydrogen outlier 
    
    if pdb_name == 'labeled_all_847.pdb.csv':
        df_pdb = replace_values(df_pdb, 104, 'H3', 0.89, -1)
#         df_pdb_w = df_pdb
    
    
    ################################
    ### 2, Check abnormal shifts ###
    ################################
    
    
    idx = np.where(df_pdb['shift'] > 150)[0]
    
    if len(idx) > 0:
        
        idx_list.extend(list(idx))
        
        atom_name_list.extend(list(df_pdb.loc[idx, :]['Atom_name'].values))
        
        mono_name_list.extend(list(df_pdb.loc[idx, :]['Residual_accurate_name'].values))
        
        shift_list.extend(list(df_pdb.loc[idx, :]['shift'].values))
        
        pdb_name_list.extend(list(np.repeat(pdb_name, len(idx))))
        pass
    
    
    ###############################################
    ### 3, create new feature for root, Me, Ser ###
    ###############################################
    
    df_pdb['Lineage'] = df_pdb['Lineage'].fillna('')
    df_pdb = check_and_reformulate_lineage(df_pdb)
    
    idx = np.where(df_pdb['Lineage'].values == '')[0][0]
    
    root_list.append(df_pdb.loc[idx, :]['Lineage'])
    initial_list.append(df_pdb.loc[idx, :]['Residual_accurate_name'])
    
#     df_pdb = create_root_effect_feature(df_pdb, root_residue = 'Me')
    
    df_pdb = create_atom_distance_feature_Me(df_pdb, pdb_name, threshold = 4)
#     df_pdb = create_atom_distance_feature_root(connection_dir, pdb_name, df_pdb, root = 'Me', threshold = 4)
    
#     df_pdb = create_root_effect_feature(df_pdb, root_residue = 'Ser')
    
    df_pdb = create_atom_distance_feature_root(connection_dir, pdb_name, df_pdb, root = 'Ser', threshold = 4)

#     if 'Me' in df_pdb['Residual_accurate_name'].values:
#         df_pdb_w = df_pdb
#         print(pdb_name)
#         me_lineage_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'Me', :]['Lineage'].values))
        
#     if 'Asn' in df_pdb['Residual_accurate_name'].values:
#         print(pdb_name)
#         asn_lineage_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'Asn', :]['Lineage'].values))
        
#     if 'Ser' in df_pdb['Residual_accurate_name'].values:
#         print(pdb_name)
#         df_pdb_w = df_pdb
#         ser_lineage_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'Ser', :]['Lineage'].values))

    
    
    ###############################################
    ### 4, create new feature for Ac, Gc, S #######
    ###############################################
    
#     if 'S' in df_pdb['Residual_accurate_name'].values:
#         print(pdb_name, i)
#         df_pdb_w = df_pdb
#         s_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'S', :]['Lineage'].values))

#     if 'Ac' in df_pdb['Residual_accurate_name'].values:
#     if 'ACY' in df_pdb['Residual_name'].values:
#         print(pdb_name, i)
#         df_pdb_w = df_pdb
#         ac_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'S', :]['Lineage'].values))
    
#     if 'Gc' in df_pdb['Residual_accurate_name'].values:
#         print(pdb_name, i)
#         df_pdb_w = df_pdb
#         ac_list.extend(list(df_pdb.loc[df_pdb['Residual_accurate_name'] == 'S', :]['Lineage'].values))
    
    
    # read in label file
    df_label_name = pdb_name.split('labeled_all_')[1].split('.pdb')[0] + '.csv'
    
    df_label_path = os.path.join(label_dir, df_label_name)
    
    df_label = pd.read_csv(df_label_path)
    
    df_label['Linkage'] = df_label['Linkage'].fillna('')
    
    # create feature
    df_pdb = identify_AC_components(df_pdb, pdb_name, df_label, connection_dir, i)
    
    df_pdb = create_atom_distance_feature_Ac(df_pdb, pdb_name, threshold = 5)
    
    df_pdb = create_atom_distance_feature_S(df_pdb, pdb_name, df_label, threshold = 5)
    
    df_pdb = create_atom_distance_feature_Gc(df_pdb, pdb_name, threshold = 5)
    
#     if np.sum('Ac' == df_label['Residue'].values) >= 2:
#         print(i)
    
#     df_pdb = create_residue_effect_feature(df_pdb, df_label, target_residue = 'S')
    
#     df_pdb = create_residue_effect_feature(df_pdb, df_label, target_residue = 'Ac')
    
#     df_pdb = create_residue_effect_feature(df_pdb, df_label, target_residue = 'Gc')
    
    
    ###################################################
    ### 5, Reformulate Ac monosaccharides: Neup #######
    ###################################################
    
    # generate few pdb, label files for testing
    for idx in range(len(df_pdb)):
        if df_pdb.loc[idx, 'Atom_name'] == 'C11':
            c11_list = list(df_pdb.loc[idx:(idx + 5), 'Atom_name'])
#             print(list(df_pdb.loc[idx:(idx + 5), 'Atom_name']))
            all_ac_list.append(c11_list)
            
            if c11_list == ['C11', 'O12', 'H2', 'H21', 'H22', 'C2']:
                df_pdb_1 = df_pdb
                df_pdb_name_1 = pdb_name
                df_label_1 = df_label
                
            if c11_list == ['C11', 'C21', 'O1', 'C1', 'O3', 'C3']:
                df_pdb_2 = df_pdb
                df_pdb_name_2 = pdb_name
                df_label_2 = df_label
                
            if c11_list == ['C11', 'C21', 'O1', 'C1', 'O3', 'C3']:
                df_pdb_3 = df_pdb
                df_pdb_name_3 = pdb_name
                df_label_3 = df_label
    
    if 'ACY' in df_pdb['Residual_name'].values:
        df_pdb_4 = df_pdb
        df_pdb_name_4 = pdb_name
        df_label_4 = df_label
#     df_pdb = reformulate_ac_monosaccarides(df_pdb, df_label, pdb_name, i)
    
    # generate S test case, so many glycans contain Sulfur components
    # Gc information is contained and only contained in the PDB
    # verify: A Sulfur contain 5 atoms, if all S labeled in 'Residual_accurate_name', 
    # then the correspondoing number of atoms divided by 5 should match with the number of s components in label file
    if 'S' in df_label['Residue'].values:
        
        num_S = len(np.where(df_label['Residue'].values == 'S')[0])
        
        df_S = df_pdb.loc[df_pdb['Residual_accurate_name'] == 'S']
        
        assert((len(df_S) / 5) == num_S)
        
#         print(num_S, i, pdb_name)
        
        df_pdb_5 = df_pdb
        df_pdb_name_5 = pdb_name
        df_label_5 = df_label
    
    # generate Gc test case, only 8 glycans contain Gc components, this is a rare components
    # Gc information is contained and only contained in the PDB
    # similar verification process as Sulfur
    if 'Gc' in df_label['Residue'].values:
        
        num_Gc = len(np.where(df_label['Residue'].values == 'Gc')[0])
#         print(num_Gc, i, pdb_name)
        
        df_Gc = df_pdb.loc[df_pdb['Residual_accurate_name'] == 'Gc']
        
        assert((len(df_Gc) / 7) == num_Gc)
        
        df_pdb_6 = df_pdb
        df_pdb_name_6 = pdb_name
        df_label_6 = df_label
        
    ###################################################
    ### 6, Create Neup features #######################
    ###################################################
    
    df_pdb = create_neup_feature(df_pdb)
    
    df_pdb = reformulate_atom_name_kdop(df_pdb, kdo_list = ['Kdo-ol', 'a-Kdop', 'b-Kdop'])
    ############################################
    ### 7, Reformulate Atom name by AC #########
    ############################################
    
    df_pdb = reformuate_atom_name_Ac(df_pdb)
    
    df_pdb = create_main_ring_shift(df_pdb)
    
    ##############################
    ### 8, Write out pdb #########
    ##############################
    
    df_pdb.to_csv(out_pdb_path, index = False)

  1%|▌                                        | 31/2399 [00:04<05:30,  7.16it/s]

labeled_all_2403.pdb.csv 30 Ac the attached monosaccharides does not contain a C11


  3%|█▏                                       | 66/2399 [00:08<03:00, 12.90it/s]

[1]
labeled_all_2066.pdb.csv 64 Ac the attached monosaccharides does not contain a C11


  3%|█▏                                       | 68/2399 [00:08<02:57, 13.15it/s]

...

 95%|█████████████████████████████████████▏ | 2289/2399 [04:42<00:14,  7.62it/s]

labeled_all_366.pdb.csv 2287 Ac the attached monosaccharides does not contain a C11


 98%|██████████████████████████████████████▍| 2363/2399 [04:51<00:03, 11.85it/s]

labeled_all_2334.pdb.csv 2363 Ac the attached monosaccharides does not contain a C11
labeled_all_2334.pdb.csv 2363 Ac the attached monosaccharides does not contain a C11
labeled_all_457.pdb.csv 2364 Ac the attached monosaccharides does not contain a C11


100%|███████████████████████████████████████| 2399/2399 [04:55<00:00,  8.11it/s]


In [25]:
df_pdb.loc[df_pdb['Ac_component'] == 1]

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust,Ac_component,Trust,bound_AB,fischer_projection_DL,origin_mono,reformulated_standard_mono,carbon_number_PF,Me_min_atom_distance,Me_min_atom_distance_threshold,Me_min_atom_path,Ser_atom_distance,Ser_atom_distance_threshold,Ser_atom_path,Ac_min_atom_distance,Ac_min_atom_distance_threshold,Ac_min_atom_path,S_min_atom_distance,S_min_atom_distance_threshold,S_min_atom_path,Gc_min_atom_distance,Gc_min_atom_distance_threshold,Gc_min_atom_path,New_Atom_name,main_ring_shift
109,HETATM,110,C11,NAG,A,7,-6.462,8.748,-5.451,C,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,83.0,a,d,glcpn,glcn,P,24,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,[C11],8,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11]",999,2,[No _Gc_ included],C11_Ac,-1.0
110,HETATM,111,C21,NAG,A,7,-5.040,8.298,-5.642,C,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,25,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21]",9,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21]",999,2,[No _Gc_ included],C21_Ac,-1.0
111,HETATM,112,O1,NAG,A,7,-6.872,9.834,-5.851,O,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,25,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, O1]",9,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, O1]",999,2,[No _Gc_ included],O1_Ac,-1.0
112,HETATM,113,H21,NAG,A,7,-4.881,7.289,-5.254,H,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,98.0,a,d,glcpn,glcn,P,26,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21, H21]",10,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21, H21]",999,2,[No _Gc_ included],H21_Ac,-1.0
113,HETATM,114,H22,NAG,A,7,-4.811,8.304,-6.711,H,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,26,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21, H22]",10,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21, H22]",999,2,[No _Gc_ included],H22_Ac,-1.0
114,HETATM,115,H23,NAG,A,7,-4.373,8.986,-5.116,H,a-D-GlcpN,"1,4,4,3,4",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,26,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21, H23]",10,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21, H23]",999,2,[No _Gc_ included],H23_Ac,-1.0
165,HETATM,166,C11,NAG,A,11,-7.233,8.537,1.706,C,a-D-GlcpN,"1,4,4,3,4,4,2",-1.0,-1,1,83.0,a,d,glcpn,glcn,P,32,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,[C11],8,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11]",999,2,[No _Gc_ included],C11_Ac,-1.0
166,HETATM,167,C21,NAG,A,11,-5.983,8.932,2.443,C,a-D-GlcpN,"1,4,4,3,4,4,2",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,33,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21]",9,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21]",999,2,[No _Gc_ included],C21_Ac,-1.0
167,HETATM,168,O1,NAG,A,11,-7.358,7.442,1.161,O,a-D-GlcpN,"1,4,4,3,4,4,2",-1.0,-1,1,NaN,a,d,glcpn,glcn,P,33,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, O1]",9,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, O1]",999,2,[No _Gc_ included],O1_Ac,-1.0
168,HETATM,169,H21,NAG,A,11,-5.516,9.792,1.955,H,a-D-GlcpN,"1,4,4,3,4,4,2",-1.0,-1,1,98.0,a,d,glcpn,glcn,P,34,2,"[C1, O1, C1, C2, C3, C4, O4, C1, C2, C3, C4, O...",999,2,[No Ser included],0,0,"[C11, C21, H21]",10,2,"[S1, O6, C6, C5, O5, C1, C2, N2, C11, C21, H21]",999,2,[No _Gc_ included],H21_Ac,-1.0


In [26]:
pdb_name

'labeled_all_1494.pdb.csv'

##### Testing parts, no main content

In [27]:
# len(pdb_files)

In [28]:
# df_pdb_name_5

In [29]:
# df_pdb_copy

In [30]:
# df_label

In [31]:
# current_target_distance_list[149]

In [32]:
# df_pdb

In [33]:
# np.array(minimum_all_Ac_distance_list_list)

In [34]:
# dict_f

In [35]:
# pd.read_csv('dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_carbon_csv/237.csv')

In [36]:
# pdb_name

In [37]:
# read_in_connection(connection_dir, pdb_name)

In [38]:
# df_pdb_1

In [39]:
# current_atom_connect, c11_atom_num, atom_num, df_label_name

In [40]:
# dict_f

In [41]:
# so many Ac patterns here, need to use connection dictionary
unique, counts = np.unique(np.array(all_ac_list), return_counts=True)

print(np.asarray((unique, counts)).T)

[[list(['C11', 'C10', 'C9', 'C8', 'C7', 'C6']) 1]
 [list(['C11', 'C12', 'C13', 'C14', 'C15', 'C16']) 7]
 [list(['C11', 'C12', 'C3', 'O3', 'H8', 'H6']) 2]
 [list(['C11', 'C12', 'C3', 'O3', 'H8', 'H7O']) 3]
 [list(['C11', 'C12', 'H', 'H1', 'H8', 'H6']) 2]
 [list(['C11', 'C12', 'H', 'H1', 'H8', 'H7O']) 1]
 [list(['C11', 'C21', 'O1', 'C1', 'O3', 'C3']) 4]
 [list(['C11', 'C21', 'O1', 'C2', 'O2', 'C3']) 3]
 [list(['C11', 'C21', 'O1', 'C3', 'O3', 'C1']) 13]
 [list(['C11', 'C21', 'O1', 'C4', 'O4', 'O5']) 3]
 [list(['C11', 'C21', 'O1', 'H21', 'H22', 'H23']) 1544]
 [list(['C11', 'C21', 'O11', 'C1', 'O1', 'O5']) 1]
 [list(['C11', 'C21', 'O11', 'C3', 'O3', 'C1']) 2]
 [list(['C11', 'C21', 'O11', 'H2', 'H21', 'H22']) 20]
 [list(['C11', 'C21', 'O11', 'H21', 'H22', 'H23']) 9]
 [list(['C11', 'C21', 'O12', 'C6', 'C7', 'O7']) 14]
 [list(['C11', 'C21', 'O12', 'H2', 'H21', 'H22']) 501]
 [list(['C11', 'C3', 'O3', 'H8', 'H6', 'H5O']) 4]
 [list(['C11', 'C3', 'O3', 'H8', 'H7O', 'H6']) 8]
 [list(['C11', 'C54', 

/tmp/ipykernel_82618/543948076.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  unique, counts = np.unique(np.array(all_ac_list), return_counts=True)


In [42]:
# df_pdb_name_1

In [43]:
# df_pdb_1.loc[df_pdb_1['Atom_name'] == 'C11']

In [44]:
df_150 = pd.DataFrame([pdb_name_list, idx_list, atom_name_list, mono_name_list, shift_list]).T
df_150.loc[(df_150[2] == 'C1')].to_csv('shift_150_C1.csv', index = False)

In [45]:
np.unique(df_150.loc[(df_150[2] == 'C1')][3].values)

array(['17HOOle', 'Ac', 'Gc', 'Kdo-ol', 'Mal', 'Ser', 'a-Kdop', 'a-Neup',
       'b-Kdop', 'b-Neup'], dtype=object)

In [46]:
# def create_neup_feature(df_pdb):

#     carbon_list = ['C1', 'C11', 'C2', 'C21', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

#     hydrogen_list = ['H1', 'H11', 'H2', 'H3', 'H31', 'H4', 'H5', 'H51', 'H6', 'H61', 'H7', 'H8', 'H9', 'H91']

#     df_pdb_copy = df_pdb.copy()
    
#     new_atom_list = []
#     for i in range(len(df_pdb)):
#         current_atom_name = df_pdb_copy.loc[i, :]['Atom_name']
#         current_residue_name = df_pdb_copy.loc[i, :]['Residual_accurate_name']

#         if ('Neup' in current_residue_name) and ((current_atom_name in carbon_list) or \
#                                                  (current_atom_name in hydrogen_list)):
#             new_atom = 'Neup_' + current_atom_name
#         else:
#             new_atom = current_atom_name
#         new_atom_list.append(new_atom)
    
#     df_pdb_copy['New_Atom_name'] = new_atom_list
    
#     return df_pdb_copy

In [47]:
# df_pdb = pd.read_csv('dataset/Godess_final_data/Godess_data/labeled_all_1417.pdb.csv')
# df_pdb_out = df_pdb.copy()

In [48]:
# df_pdb_out.loc[(df_pdb_out['Residual_accurate_name'] == 'b-D-ManpA') & 
#                            (df_pdb_out['trust'] == 69) & 
#                            (df_pdb_out['Atom_name'] == 'C2') & 
#                            (df_pdb_out['shift'] == 70.6), ['shift']]

In [49]:
# df_pdb

In [50]:
# df_pdb_w

In [51]:
# a = pd.read_csv('dataset/Godess_final_data/visulize/carbon/train_carbon.csv').sort_values(['1'], ascending = False, ignore_index=True)

# A = a.loc[a['atom_type'] == 'C1']

In [52]:
# matching_table = pd.read_csv('new_matching_table.csv')
# matching_table.loc[matching_table['2'] == 1591]

# matching_table.loc[matching_table['2'] == 888]

# matching_table.loc[matching_table['2'] == 1417]

##### old formula, test codes

In [53]:
2, # # old formula, identify Ac monosacchaide by atom pattern H21, H22, H23
# def reformulate_ac_monosaccarides(df_pdb, df_label, pdb_name, j):

#     df_pdb_copy = df_pdb.copy()

#     for i in range(len(df_label)):
#         current_linkage = df_label.loc[i, :]['Linkage']

#         if isinstance(current_linkage, float):
#             current_linkage = int(current_linkage)
#             current_linkage = str(current_linkage)

#         current_residue = df_label.loc[i, :]['Residue']

#         if (current_residue == 'Ac'):
#             attached_linkage = current_linkage[0:-2]

#             df_attached_linkage = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage]

            
#             if df_attached_linkage.empty:
                
#                 print(pdb_name, j)
#                 continue

#             c11_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'C11'].index
#             c21_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'C21'].index
#             o1_idx = c21_idx + 1
#             h21_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H21'].index
#             h22_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H22'].index
#             h23_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H23'].index
            
#             h2_neup_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H2'].index
#             h21_neup_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H21'].index
#             h22_neup_idx = df_attached_linkage.loc[df_attached_linkage['Atom_name'] == 'H22'].index
            
#             if ((c21_idx - c11_idx) == (h21_idx - o1_idx) == (h22_idx - h21_idx) == (h23_idx - h22_idx)) and \
#                 ('O' in df_attached_linkage.loc[o1_idx, :]['Atom_name'].values[0]):
                

# #                 df_pdb_copy.loc[c11_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[c11_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[c11_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[c11_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[c11_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[c11_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[c11_idx, ['Ac_effect']] = 'Ac'


# #                 df_pdb_copy.loc[c21_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[c21_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[c21_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[c21_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[c21_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[c21_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[c21_idx, ['Ac_effect']] = 'Ac'

# #                 df_pdb_copy.loc[o1_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[o1_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[o1_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[o1_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[o1_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[o1_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[o1_idx, ['Ac_effect']] = 'Ac'

# #                 df_pdb_copy.loc[h21_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[h21_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[h21_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[h21_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[h21_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[h21_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[h21_idx, ['Ac_effect']] = 'Ac'

# #                 df_pdb_copy.loc[h22_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[h22_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[h22_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[h22_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[h22_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[h22_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[h22_idx, ['Ac_effect']] = 'Ac'

# #                 df_pdb_copy.loc[h23_idx, ['Residual_accurate_name']] = 'Ac'
# #                 df_pdb_copy.loc[h23_idx, ['bound_AB']] = 'missing_a_b'
# #                 df_pdb_copy.loc[h23_idx, ['fischer_projection_DL']] = 'missing_L_D'
# #                 df_pdb_copy.loc[h23_idx, ['origin_mono']] = 'ac'
# #                 df_pdb_copy.loc[h23_idx, ['reformulated_standard_mono']] = 'ac'
# #                 df_pdb_copy.loc[h23_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                 df_pdb_copy.loc[h23_idx, ['Ac_effect']] = 'Ac'
    
#             elif ((c21_idx - c11_idx) == (h2_neup_idx - o1_idx) == (h21_neup_idx - h2_neup_idx) == (h22_neup_idx - h21_neup_idx)) and \
#                     ('O' in df_attached_linkage.loc[o1_idx, :]['Atom_name'].values[0]):


#     #                 df_pdb_copy.loc[c11_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[c11_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[c11_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[c11_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[c11_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[c11_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[c11_idx, ['Ac_effect']] = 'Ac'


#     #                 df_pdb_copy.loc[c21_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[c21_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[c21_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[c21_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[c21_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[c21_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[c21_idx, ['Ac_effect']] = 'Ac'

#     #                 df_pdb_copy.loc[o1_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[o1_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[o1_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[o1_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[o1_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[o1_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[o1_idx, ['Ac_effect']] = 'Ac'

#     #                 df_pdb_copy.loc[h21_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[h21_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[h21_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[h21_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h21_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h21_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[h2_neup_idx, ['Ac_effect']] = 'Ac'

#     #                 df_pdb_copy.loc[h22_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[h22_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[h22_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[h22_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h22_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h22_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[h21_neup_idx, ['Ac_effect']] = 'Ac'

#     #                 df_pdb_copy.loc[h23_idx, ['Residual_accurate_name']] = 'Ac'
#     #                 df_pdb_copy.loc[h23_idx, ['bound_AB']] = 'missing_a_b'
#     #                 df_pdb_copy.loc[h23_idx, ['fischer_projection_DL']] = 'missing_L_D'
#     #                 df_pdb_copy.loc[h23_idx, ['origin_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h23_idx, ['reformulated_standard_mono']] = 'ac'
#     #                 df_pdb_copy.loc[h23_idx, ['carbon_number_PF']] = 'missing_carbon_number'
#                     df_pdb_copy.loc[h22_neup_idx, ['Ac_effect']] = 'Ac'


#     return df_pdb_copy

In [54]:
# dict_f = read_in_connection(connection_dir, df_pdb_name_4)

# df_pdb_copy = df_pdb_4.copy()

# df_label = df_label_4

# Ac_components_list = np.repeat(0, len(df_pdb_copy))

# for i in range(len(df_label)):
#     current_linkage = df_label.loc[i, :]['Linkage']

#     if isinstance(current_linkage, float):
#         current_linkage = int(current_linkage)
#         current_linkage = str(current_linkage)

#     current_residue = df_label.loc[i, :]['Residue']
    
#     if current_residue == 'Ac':
        
#         attached_linkage = current_linkage[0:-2]

#         attached_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage].index
        
#         attached_atom_name = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage, :]['Atom_name'].values
        
#         if 'C11' in attached_atom_name:
#             c11_index_list = np.where(attached_atom_name == 'C11')[0]
            
#             # Attached only one monosaccharide
#             if len(c11_index_list) == 1:

#                 # c11 index in csv
#                 c11_index = attached_index[c11_index_list[0]]
                
#                 #c11 index in atom names
#                 c11_atom_num = df_pdb_copy.loc[c11_index, :]['Atom_num']
                
#                 c11_connection_list = dict_f[c11_atom_num]
                
#                 Ac_residue_index_list = [c11_atom_num]
                
#                 for atom_num in c11_connection_list:
                    
#                     current_atom_connect = dict_f[atom_num]
                    
#                     current_atom_connect.remove(c11_atom_num)
                    
#                     # identify the Oxx
#                     if len(current_atom_connect) == 0:
                        
#                         Ac_residue_index_list.append(atom_num)
                    
#                     else:
#                         len_atom_connect = 0
#                         for atom_num_connect in current_atom_connect:
                            
                            
#                             atom_num_connect_atom_list = dict_f[atom_num_connect]
                            
#                             atom_num_connect_atom_list.remove(atom_num)
# #                             print(atom_num_connect, atom_num_connect_atom_list, atom_num)
    
#                             len_atom_connect += len(atom_num_connect_atom_list)
                        
#                         # identify the C21, and three HXXs 
#                         if len_atom_connect == 0:
                            
#                             Ac_residue_index_list.append(atom_num)
#                             Ac_residue_index_list.extend(dict_f[atom_num])
                            
#                 Ac_residue_index_list = df_pdb_copy.loc[df_pdb_copy['Atom_num'].isin(Ac_residue_index_list)].index
#                 Ac_components_list[Ac_residue_index_list] = 1
# #                     print(atom_num, current_atom_connect)
#             else:
#                 print(df_pdb_name_4, i, 'contain more than one C11')
            
            
#         else:
            
#             print(df_pdb_name_4, i, current_residue, 'the attached monosaccharides does not contain a C11')
            
#             current_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == current_linkage].index
            
#             current_three_letter_residual = df_pdb_copy.loc[current_index, :]['Residual_name'].values
            
#             # the attached monosaccharide 
#             assert(np.all(current_three_letter_residual == 'ACY'))
            
# # some ACs are behave like are separate components
# if not df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].empty:
#     ACY_index = df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].index
#     Ac_components_list[ACY_index] = 1

# df_pdb_copy['Ac_component'] = Ac_components_list

In [55]:
# threshold = 5

# df_pdb_copy = df_pdb_1.copy()

# dict_f = read_in_connection(connection_dir, df_pdb_name_1)

# df_c11 = df_pdb_copy.loc[(df_pdb_copy['Ac_component'] == 1) & 
#                          ((df_pdb_copy['Atom_name'] == 'C11') | 
#                           (df_pdb_copy['Atom_name'] == 'C1'))]

# df_c11.index = range(len(df_c11))

# # since there could be multiple Acs in a glycan, each glycan corresponds to a glycan list
# # therefore, need to first retrieve list of glycan distance list

# all_Ac_distance_list_list = []

# all_Ac_distance_threshold_list_list = []

# all_Ac_atom_name_list_list_list = []

# all_atom_num_list = df_pdb_copy['Atom_num'].values

# all_atom_name_list = df_pdb_copy['Atom_name'].values

# for i in range(len(df_c11)):
    
#     current_target_distance_list = []
    
#     current_target_atom_name_list_list = []
    
#     current_atom_num = df_c11.loc[i, :]['Atom_num']
    
#     for j in all_atom_num_list:
        
#         current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)
        
#         current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]
        
#         # 3, list of list of atom paths, index + 1 = atom num
#         current_target_atom_name_list_list.append(current_atom_name_path)
        
#         # 2, Assign distance 
#         if current_atom_num == j:
            
#             current_target_distance_list.append(0)
        
#         # atom with in the same Ac component labeled as 0
#         elif ((len(current_atom_num_path) - 1) <= 2) and (df_pdb_copy.loc[j-1, :]['Ac_component'] == 1):
             
#             current_target_distance_list.append(0)
        
#         else:
#             current_target_distance_list.append(len(current_atom_num_path) - 1)
            
#     # 1, Assign categorical value by threshold as distance bound
#     current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

#     current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

#     current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2
    
#     current_target_distance_list_threshold = list(current_target_distance_list_threshold)

#     # list of list stores distance to each Acs in a glycan
# #     print(current_target_distance_list)
#     all_Ac_distance_list_list.append(current_target_distance_list)
    
#     # this is not used in this function, but may be used in the future if other method is developed
#     all_Ac_distance_threshold_list_list.append(current_target_distance_list_threshold)

#     all_Ac_atom_name_list_list_list.append(current_target_atom_name_list_list)
    
# all_Ac_distance_list_list = np.array(all_Ac_distance_list_list)

# # find the minimum distance
# new_all_index = all_Ac_distance_list_list.argmin(axis=0)

# minimum_all_Ac_distance_list_list = [all_Ac_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

# # find the corresponding minimum threshold
# minimum_all_Ac_distance_list_list_threshold = np.array(minimum_all_Ac_distance_list_list.copy())

# minimum_all_Ac_distance_list_list_threshold[(minimum_all_Ac_distance_list_list_threshold <= threshold) & (minimum_all_Ac_distance_list_list_threshold > 0)] = 1

# minimum_all_Ac_distance_list_list_threshold[minimum_all_Ac_distance_list_list_threshold > threshold] = 2

# minimum_all_Ac_distance_list_list_threshold = list(minimum_all_Ac_distance_list_list_threshold)

# # find the corresponding minimum atom names

# minimum_all_Ac_atom_name_list_list_list = [all_Ac_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

In [56]:
# df_pdb_copy = df_pdb_5.copy()

# dict_f = read_in_connection(connection_dir, df_pdb_name_5)

# df_label = df_label_5

# threshold = 5

# df_s = df_pdb_copy.loc[(df_pdb_copy['Residual_name'] == 'SO4') & 
#                          (df_pdb_copy['Atom_type'] == 'S')]
# df_s.index = range(len(df_s))

# if (not df_s.empty) and ('S' in df_label['Residue'].values):

#     all_S_distance_list_list = []

#     all_S_distance_threshold_list_list = []

#     all_S_atom_name_list_list_list = []

#     all_atom_num_list = df_pdb_copy['Atom_num'].values

#     all_atom_name_list = df_pdb_copy['Atom_name'].values
    
#     for i in range(len(df_s)):

#         current_target_distance_list = []

#         current_target_atom_name_list_list = []

#         current_atom_num = df_s.loc[i, :]['Atom_num']
        
#         for j in all_atom_num_list:

#             current_atom_num_path = bfs_shortest_path(dict_f, current_atom_num, j)

#             current_atom_name_path = [all_atom_name_list[k-1] for k in current_atom_num_path]

#             # 3, list of list of atom paths, index + 1 = atom num
#             current_target_atom_name_list_list.append(current_atom_name_path)

#             # 2, Assign distance 
#             if current_atom_num == j:

#                 current_target_distance_list.append(0)

#             # atom with in the same S component labeled as 0
#             elif ((len(current_atom_num_path) - 1) <= 2) and (df_pdb_copy.loc[j-1, :]['Residual_accurate_name'] == 'S'):

#                 current_target_distance_list.append(0)

#             else:
#                 current_target_distance_list.append(len(current_atom_num_path) - 1)
                
        
#          # 1, Assign categorical value by threshold as distance bound
#         current_target_distance_list_threshold = np.array(current_target_distance_list.copy())

#         current_target_distance_list_threshold[(current_target_distance_list_threshold <= threshold) & (current_target_distance_list_threshold > 0)] = 1

#         current_target_distance_list_threshold[current_target_distance_list_threshold > threshold] = 2

#         current_target_distance_list_threshold = list(current_target_distance_list_threshold)
        
#         # list of list stores distance to each Ss in a glycan
# #         print(current_target_distance_list)

#         all_S_distance_list_list.append(current_target_distance_list)

#         # this is not used in this function, but may be used in the future if other method is developed
#         all_S_distance_threshold_list_list.append(current_target_distance_list_threshold)

#         all_S_atom_name_list_list_list.append(current_target_atom_name_list_list)
    
    
#     all_S_distance_list_list = np.array(all_S_distance_list_list)

#     # find the minimum distance
#     new_all_index = all_S_distance_list_list.argmin(axis=0)

#     minimum_all_S_distance_list_list = [all_S_distance_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]

#     # find the corresponding minimum threshold
#     minimum_all_S_distance_list_list_threshold = np.array(minimum_all_S_distance_list_list.copy())

#     minimum_all_S_distance_list_list_threshold[(minimum_all_S_distance_list_list_threshold <= threshold) & (minimum_all_S_distance_list_list_threshold > 0)] = 1

#     minimum_all_S_distance_list_list_threshold[minimum_all_S_distance_list_list_threshold > threshold] = 2

#     minimum_all_S_distance_list_list_threshold = list(minimum_all_S_distance_list_list_threshold)

#     # find the corresponding minimum atom names

#     minimum_all_S_atom_name_list_list_list = [all_S_atom_name_list_list_list[new_all_index[m]][m] for m in range(len(new_all_index))]


#     df_pdb_copy['S' + '_min_atom_distance'] = minimum_all_S_distance_list_list

#     df_pdb_copy['S' + '_min_atom_distance_threshold'] = minimum_all_S_distance_list_list_threshold

#     df_pdb_copy['S' + '_min_atom_path'] = minimum_all_S_atom_name_list_list_list
    
# else:
#     # not containing any S components
#     df_pdb_copy['S' + '_min_atom_distance'] = 999

#     df_pdb_copy['S' + '_min_atom_distance_threshold'] = 2

#     df_pdb_copy['S' + '_min_atom_path'] = [['No ' + '_S_' + ' included'] for _ in range(len(df_pdb))]